In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np

In [2]:
london_data = pd.read_csv("../Binary problem/Data/dataset_london.csv")

london_with_driving_lic = london_data[london_data['driving_license']==1]

london_with_car = london_with_driving_lic[london_with_driving_lic['car_ownership']>0]

ld_with_one_trip = london_with_car.groupby(by=['household_id']).sample(1)

Feature = ld_with_one_trip[['travel_mode', 'survey_year', 'dur_walking', 'dur_cycling', 'dur_pt_total', 'dur_driving', 'cost_transit', 'cost_driving_total']]

X = Feature[['survey_year', 'dur_walking', 'dur_cycling', 'dur_pt_total', 'cost_transit', 'dur_driving', 'cost_driving_total']]
y = Feature[['travel_mode', 'survey_year']]


new_mode = {'drive':0, 'pt':1, 'cycle':2,'walk':3}

y = y.replace({'travel_mode': new_mode})


X_train = X[X['survey_year']<3]
X_validate =  X[X['survey_year']==3]
y_train = y[y['survey_year']<3]
y_validate = y[y['survey_year']==3]

X_train = X_train.drop(columns=['survey_year'])
X_validate = X_validate.drop(columns=['survey_year'])
y_train = y_train.drop(columns=['survey_year'])
y_validate = y_validate.drop(columns=['survey_year'])

In [3]:
train_data = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
validation_data = lgb.Dataset(X_validate, label=y_validate, reference= train_data, free_raw_data=False)

In [4]:
param_ld_unc = {'max_depth': 1, 
         'num_boost_round': 5, 
         'objective':'multiclass',
         'learning_rate': .1,
         'verbosity': 2,
         'num_classes': 4,
         'min_data_in_leaf': 1,
         'min_sum_hessian_in_leaf':0,
         'linear_trees':True
        }

In [5]:
lightgbm_ld_unc = lgb.train(param_ld_unc, train_data)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.042517
[LightGBM] [Debug] init for col-wise cost 0.000034 seconds, init for row-wise cost 0.001797 seconds
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1354
[LightGBM] [Info] Number of data points in the train set: 7103, number of used features: 6
[LightGBM] [Info] Start training from score -0.460225
[LightGBM] [Info] Start training from score -1.486148
[LightGBM] [Info] Start training from score -3.692123
[LightGBM] [Info] Start training from score -2.139644
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth =

c:\Users\nicol\Anaconda3\envs\lightgbm\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [6]:
lightgbm_ld_unc.predict(X_train.iloc[3:5])

array([[0.69085928, 0.18938749, 0.0263643 , 0.09338893],
       [0.67324876, 0.22903049, 0.01929918, 0.07842156]])

In [7]:
X_train.iloc[3:5]

p1 0.71  g1 0.29 h1 0.2059
p2 0
p3 0
p4 0.29  g4 -0.29 h4 

SyntaxError: invalid syntax (2814738559.py, line 3)

In [65]:
y_train.iloc[:5]

,travel_mode
2,0
11,0
14,0
20,0
25,3


In [ ]:
'leaf_value': 0.05685279708831137
'leaf_weight': 0.3333333432674408

In [14]:
lightgbm_ld_unc.predict(X_validate)

array([[0.68250394, 0.19534773, 0.02637097, 0.09577736],
       [0.52205472, 0.12248715, 0.01995932, 0.3354988 ],
       [0.5875498 , 0.13601077, 0.02248831, 0.25395113],
       ...,
       [0.67048564, 0.17530552, 0.02577123, 0.1284376 ],
       [0.67921164, 0.20306016, 0.02655518, 0.09117301],
       [0.56370429, 0.35148067, 0.01653775, 0.06827729]])

In [20]:
model = lightgbm_ld_unc.dump_model()
model

{'name': 'tree',
 'version': 'v3',
 'num_class': 4,
 'num_tree_per_iteration': 4,
 'label_index': 0,
 'max_feature_idx': 5,
 'objective': 'multiclass num_class:4',
 'average_output': False,
 'feature_names': ['dur_walking',
  'dur_cycling',
  'dur_pt_total',
  'cost_transit',
  'dur_driving',
  'cost_driving_total'],
 'monotone_constraints': [],
 'feature_infos': {'dur_walking': {'min_value': 0.025555556,
   'max_value': 8.804444444,
   'values': []},
  'dur_cycling': {'min_value': 0.009444444,
   'max_value': 3.051944444,
   'values': []},
  'dur_pt_total': {'min_value': 0.013055556,
   'max_value': 2.735277778,
   'values': []},
  'cost_transit': {'min_value': 0, 'max_value': 11.7, 'values': []},
  'dur_driving': {'min_value': 0.008611110999999998,
   'max_value': 1.727777778,
   'values': []},
  'cost_driving_total': {'min_value': 0.02, 'max_value': 15.67, 'values': []}},
 'tree_info': [{'tree_index': 0,
   'num_leaves': 2,
   'num_cat': 0,
   'shrinkage': 1,
   'tree_structure': {'

In [51]:
new_p1 = np.exp(-0.5*0.3817384822141626)
new_p2 = np.exp(-0.5*1.6746763792421082)
new_p3 = np.exp(-0.5*3.7242164796099657)
new_p4 = np.exp(-0.5*0.5907802951783105)

new_sum = new_p1 + new_p2 + new_p3 + new_p4

newp1 = new_p1/new_sum
newp2 = new_p2/new_sum
newp3 = new_p3/new_sum
newp4 = new_p4/new_sum

newp1

0.38275133506085757

In [45]:
p4

0.11924539168303395

In [81]:
summed = np.exp(-0.693147) + np.exp(-34.538776) + np.exp(-34.538776) + np.exp(-0.693147)
p1 = np.exp(-0.693147)/summed
p2 = np.exp(-34.538776)/summed
p3 = np.exp(-34.538776)/summed
p4 = np.exp(-0.693147)/summed

In [82]:
[p1, p2, p3, p4]

[0.499999999999999,
 1.0000002143507624e-15,
 1.0000002143507624e-15,
 0.499999999999999]

In [ ]:
g1 = 

In [34]:
preds = np.repeat([[p1], [p2], [p3], [p4]], 7103, axis = 1).T

In [35]:
preds

array([[0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       ...,
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539],
       [0.63663245, 0.22047019, 0.02365197, 0.11924539]])

In [36]:
c_entr = 0
for i, l in enumerate(y_train['travel_mode']):
    c_entr += np.log(preds[i, l])


In [110]:
labels1 = (y_train).astype(int)

In [155]:
g1 = labels1-p1
h1 = np.repeat(p1 * (1 - p1), 7103)

In [156]:
g_split1 = g1[X_train['cost_driving_total']<7.070000000000001]
h_split1 = h1[X_train['cost_driving_total']<7.070000000000001]
g_split1 = g_split1['travel_mode'].tolist()
h_split1 = h_split1.tolist()
g_split2 = g1[X_train['cost_driving_total']>7.070000000000001]
h_split2 = h1[X_train['cost_driving_total']>7.070000000000001]
g_split2 = g_split2['travel_mode'].tolist()
h_split2 = h_split2.tolist()



In [157]:
np.size(g_split1)

6381

In [172]:
np.sum(g1)**2/(722+6381)

travel_mode    1.552188
dtype: float64

In [173]:
(np.sum(g_split1)**2/6381 + np.sum(g_split2)**2/722)

165.21321820703662

In [161]:
weights = [g^2/h for g, h in zip(g_split1, h_split1)]

TypeError: unsupported operand type(s) for ^: 'float' and 'float'

In [162]:
h_split1

[0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.23428250467547967,
 0.2342825

In [163]:
np.mean(weights)

-0.15484219236329722

In [165]:
np.sum(h_split1)

1494.9566623342357

In [82]:
h_split2

[0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515,
 0.21759997627616515]

In [18]:
data_dict = {'x_0': [3, 5, 1],
             'x_1': [40, 2, -21],
             'y': [0, 1, 2]}

data_df = pd.DataFrame(data_dict)

train_data = lgb.Dataset(data_df[['x_0', 'x_1']], label=data_df[['y']], free_raw_data=False)

param = {'max_depth': 1, 
         'num_boost_round': 2, 
         'objective':'multiclass',
         'learning_rate': 1,
         'verbosity': 2,
         'num_classes': 3,
         'min_data_in_leaf': 1,
         'min_sum_hessian_in_leaf':0,
         'reg_alpha':0,
         'reg_lambda':0
        }

model = lgb.train(param, train_data)

[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.000000
[LightGBM] [Debug] init for col-wise cost 0.000017 seconds, init for row-wise cost 0.000831 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 3
[LightGBM] [Info] Number of data points in the train set: 3, number of used features: 1
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Debug] Trained a tree with leaves = 2 and depth = 1
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

c:\Users\nicol\Anaconda3\envs\lightgbm\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [19]:
model_dump = model.dump_model()
model_dump

{'name': 'tree',
 'version': 'v3',
 'num_class': 3,
 'num_tree_per_iteration': 3,
 'label_index': 0,
 'max_feature_idx': 1,
 'objective': 'multiclass num_class:3',
 'average_output': False,
 'feature_names': ['x_0', 'x_1'],
 'monotone_constraints': [],
 'feature_infos': {'x_1': {'min_value': -21, 'max_value': 40, 'values': []}},
 'tree_info': [{'tree_index': 0,
   'num_leaves': 2,
   'num_cat': 0,
   'shrinkage': 1,
   'tree_structure': {'split_index': 0,
    'split_feature': 1,
    'split_gain': 0.5,
    'threshold': 1.0000000180025095e-35,
    'decision_type': '<=',
    'default_left': True,
    'missing_type': 'None',
    'internal_value': -1.09861,
    'internal_weight': 0,
    'internal_count': 3,
    'left_child': {'leaf_index': 0,
     'leaf_value': -2.098612288668107,
     'leaf_weight': 0.3333333432674408,
     'leaf_count': 1},
    'right_child': {'leaf_index': 1,
     'leaf_value': -0.5986122886681104,
     'leaf_weight': 0.6666666865348816,
     'leaf_count': 2}}},
  {'tree